<h1>Network Analysis - Extracting Features from Network Graphs</h1>
<b>Éverton Bin</b><br>

<h3>Introduction</h3>
<p>
    Through this notebook, we are going to extract features from a company's e-mail network, where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people. The e-mail network can be found in the file <i>email_prediction.txt</i>.
</p>
<p>
    The network also contains the node attributes <i>Department</i> and <i>ManagementSalary</i>. <i>Department</i> indicates the department in the company which the person belongs to, and <i>ManagementSalary</i> indicates whether that person is receiving a management position salary.
</p>
<p>
    <b>Part 1 - Salary Prediction</b>
</p>
<p>
   Using the given network, we are going to identify the people in the network with missing values for the node attribute <i>ManagementSalary</i> and predict whether or not these individuals are receiving a management position salary.
</p>
<p>
    <b>Part 2 - New Connections Prediction</b>
</p>
<p>
    For Part 2, we will predict future connections between employees of the network. The future connections information will be loaded into the variable <i>future_connections</i>. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the <i>Future Connection</i> column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.
</p>
<p>
    Using the given network and <i>future_connections</i>, we will identify the edges in <i>future_connections</i> with missing values and predict whether or not these edges will have a future connection.
</p>
<p>
    This data was made available by <a href = "https://umich.edu/">University of Michigan</a> along with the <a href = "https://www.coursera.org/learn/python-social-network-analysis">Applied Social Network Analysis in Python</a> course offered through the <b>Coursera</b> platform, and it can only be accessed in the platform.
</p>
<p>

In [1]:
# Loading libraries:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [2]:
# Loading company's email network:
G = nx.read_gpickle('email_prediction.txt')

# Checking network:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


<p>
    We can see the network represents a company with at least 1005 employees (nodes) that have exchanged at least 16706 e-mails (edges) with each other.
</p>

<h3>Part 1 - Salary Prediction</h3>

<p>
    First, we are going to create a pandas dataframe, using the network's nodes as index and extracting the attributes <i>Department</i> and <i>ManagementSalary</i> as features:
</p>

In [3]:
# Creating dataframe with nodes as index:
df_node = pd.DataFrame(index = G.nodes())

# Extracting Department and ManagementSalary as features:
df_node['department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
df_node['mng_salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
df_node.head()

,department,mng_salary
0,1,0.0
1,1,NaN
2,21,NaN
3,21,1.0
4,21,1.0


<p>
    Next step, we are going to use different centrality measures for networks as features:
    <li><b>Degree Centrality</b>: it assumes that important nodes have many connections.</li>
    <li><b>Closeness Centrality</b>: it assumes that important nodes are the ones that are close to the others.</li>
    <li><b>Betweenness Centrality</b>: it assumes that important nodes connect other nodes.</li>
    <li><b>PageRank</b>: it assigns a score of importance for each node, given its in-link proportion (for directed graphs). In our case, it is going to be based on simple node's degree.</li>
    <li><b>HITS</b>: it assigns Hub and Authority scores, giving an idea of importance to the nodes according to its connections.</li>
</p>

In [4]:
# Including centrality measures:
# Degree Centrality:
df_node['degree_cent'] = pd.Series(nx.degree_centrality(G))

# Closeness Centrality:
df_node['closeness_cent'] = pd.Series(nx.closeness_centrality(G, normalized = True))

# Betweenness Centrality:
df_node['btwn_cent'] = pd.Series(nx.betweenness_centrality(G, normalized = True, endpoints = False))

# PageRank score:
df_node['pagerank'] = pd.Series(nx.pagerank(G, alpha = 0.85))

# Hub and authority scores:
hits = nx.hits(G)
df_node['hub'] = pd.Series(hits[0])
df_node['auth'] = pd.Series(hits[1])

df_node.head()

,department,mng_salary,degree_cent,closeness_cent,btwn_cent,pagerank,hub,auth
0,1,0.0,0.043825,0.421991,0.001124,0.001224,0.000944,0.000944
1,1,NaN,0.051793,0.422360,0.001195,0.001426,0.001472,0.001472
2,21,NaN,0.094622,0.461490,0.006570,0.002605,0.002680,0.002680
3,21,1.0,0.070717,0.441663,0.001654,0.001833,0.002369,0.002369
4,21,1.0,0.095618,0.462152,0.005547,0.002526,0.003055,0.003055


<p>
    After extracting and creating new features, we are going to split the dataframe into two parts: one with information about Management Salary, and other with missing Management Salary information, which we want to predict.
</p>

In [5]:
# Selecting observations with missing and not missing ManagementSalary information:
df_pred = df_node[df_node['mng_salary'].isnull()]
df = df_node[df_node['mng_salary'].isnull() == False]

In [6]:
# Selecting features and target:
feat_columns = ['department', 'degree_cent', 'closeness_cent', 'btwn_cent', 'pagerank', 'hub', 'auth']
X = df[feat_columns]
y = df['mng_salary']

<p>
    With the dataframe splitted into predictive and target features, we are going to split it into train and test sets.
</p>
<p>
    After that, using train set, we are going to use <i>GridSearchCV</i> to look for best parameters to train a <i>Random Forest Classifier</i> model:
</p>

In [7]:
# Splitting data into train and test sets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 101)

# Using GreadSearchCV to find the best parameters for the Random Forest Classifier:
rand_for = RandomForestClassifier()
grid_values = {'n_estimators': [15, 30, 50, 75], 'max_features': [2, 3, 4, 5], 'max_depth': [None, 2, 3, 4, 5]}
grid_rand_for_auc = GridSearchCV(rand_for, param_grid = grid_values, scoring = 'roc_auc')
grid_rand_for_auc.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [15, 30, 50, 75], 'max_features': [2, 3, 4, 5], 'max_depth': [None, 2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [8]:
# Best ROC score:
grid_rand_for_auc.best_score_

0.90431590058241273

In [9]:
# Best parameters:
grid_rand_for_auc.best_params_

{'max_depth': None, 'max_features': 2, 'n_estimators': 75}

<p>
    Using the GridSearchCV best parameters, we will finally train the Random Forest model in order to predict Management Salary for the employees with missing salary information.
<p>

In [10]:
# Training Random Forest model with best parameters:
random_forest = RandomForestClassifier(max_depth = 5, max_features = 4, n_estimators = 50, random_state = 100)
random_forest.fit(X_train, y_train)

# Evaluating the model with test set:
y_pred_test = random_forest.predict(X_test)
roc_auc = roc_auc_score(y_test, y_pred_test)

print('Roc AUC score for test set: '+ str(round(roc_auc, 2)))

Roc AUC score for test set: 0.7


In [11]:
# Making probability prediction for missing ManagementSalary:
X_pred = df_pred[feat_columns]
y_pred = random_forest.predict_proba(X_pred)

# Creating a pandas Series indicating the employee's probability of having a Management Salary:
prob_list = [prob[1] for prob in y_pred]
mng_salary_prob = pd.Series(data = prob_list, index = df_pred.index.values)

mng_salary_prob

1       0.122229
2       1.000000
5       1.000000
8       0.088217
14      0.101852
18      0.198015
27      0.127553
30      0.219130
31      0.367688
34      0.032050
37      0.044272
40      0.132729
45      0.127597
54      0.341600
55      0.168802
60      0.138372
62      1.000000
65      0.966667
77      0.078712
79      0.369308
97      0.019452
101     0.029165
103     0.178275
108     0.131455
113     0.153580
122     0.017881
141     0.275077
142     0.980000
144     0.090000
145     0.118164
          ...   
913     0.016159
914     0.028963
915     0.006407
918     0.024498
923     0.003262
926     0.016671
931     0.016671
934     0.008348
939     0.003797
944     0.010387
945     0.003797
947     0.100891
950     0.252931
951     0.004833
953     0.016159
959     0.004836
962     0.003797
963     0.248445
968     0.006630
969     0.006661
974     0.006804
984     0.008348
987     0.007959
989     0.005059
991     0.022170
992     0.005059
994     0.010387
996     0.0064

<h3>Part 2 - New Coonections Prediction</h3>

<p>
    First, we are going to load the <i>Future Connections</i> file that indicates whether the nodes connected or not, or if the information is missing.
</p>

In [12]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


<p>
    As we did in Part 1, we will now create a dataframe with information about the edges, instead of nodes.
</p>
<p>
    Since we are interested in future connections, our dataframe will have, as index, all the possible edges (e-mail link between employees) that haven't happened yet. For that, we use <b>non_edges</b> function:
</p>

In [13]:
# Creating dataframe for the edges:
df_edges = pd.DataFrame(index=nx.non_edges(G))

<p>
    After that, we are going to use link measures as features:
</p>
    <li><b>Common Neighbors</b>: the number of common neighbors define whether two nodes are likely to connect.</li>
    <li><b>Jaccard Coefficient</b>: it is the common neighbors normalized by the total number of neighbors between the two nodes.</li>
    <li><b>Resource Allocation</b>: indicates the fraction of 'resource' that one node can send to another through their common neighbors - it penalizes common neighbors that have lots of neighbors themselves.</li>
    <li><b>Preferential Attachment</b>: it assumes that nodes with high degree are more likely to get more neighbors.</li>
</p>

In [14]:
# Creating edge based features:
# Common Neighbors:
common_neigh = [(e[0], e[1], len(list(nx.common_neighbors(G, e[0], e[1])))) for e in df_edges.index]
df_edges['common_neigh'] = [cn[2] for cn in common_neigh]

# Jaccard Coefficient
df_edges['jacc'] = [i[2] for i in nx.jaccard_coefficient(G, df_edges.index)]

# Resource Allocation:
df_edges['res_alloc'] = [i[2] for i in nx.resource_allocation_index(G, df_edges.index)]

# Preferential Attachment:
df_edges['pref_attach'] = [i[2] for i in nx.preferential_attachment(G, df_edges.index)]

df_edges.head()

,common_neigh,jacc,res_alloc,pref_attach
"(0, 2)",6,0.045802,0.055340,4180
"(0, 3)",3,0.027273,0.021388,3124
"(0, 4)",3,0.022222,0.021388,4224
"(0, 7)",4,0.036364,0.061668,3168
"(0, 8)",1,0.012821,0.011628,1628


<p>
    Now, we are going to merge the edge dataframe with the future_connections dataframe by index:
</p>

In [15]:
# Merging edges dataframes into one:
df_edges = df_edges.merge(future_connections, left_index = True, right_index = True)
df_edges.head()

,common_neigh,jacc,res_alloc,pref_attach,Future Connection
"(0, 2)",6,0.045802,0.055340,4180,0.0
"(0, 3)",3,0.027273,0.021388,3124,0.0
"(0, 4)",3,0.022222,0.021388,4224,0.0
"(0, 7)",4,0.036364,0.061668,3168,0.0
"(0, 8)",1,0.012821,0.011628,1628,0.0


<p>
    As we did before, we are going to separate observations with missing future connections information from the others.
</p>
<p>
    After that, we are going to split data into train and test sets:
</p>

In [16]:
# Splitting dataframe:
df_e = df_edges[df_edges['Future Connection'].isnull() == False]
df_e_pred = df_edges[df_edges['Future Connection'].isnull()]

# Creating train and test sets:
e_feat_columns = ['common_neigh', 'jacc', 'res_alloc', 'pref_attach']
X_e = df_e[e_feat_columns]
y_e = df_e['Future Connection']

X_e_train, X_e_test, y_e_train, y_e_test = train_test_split(X_e, y_e, test_size = .2, random_state = 201)

<p>
    With <i>GridSearchCV</i>, we are going to search for the best parameters for this new model:
</p>

In [17]:
# Using GreadSearchCV to find the best parameters for the Random Forest Classifier:
rand_for = RandomForestClassifier(random_state = 300)
grid_values = {'n_estimators': [15, 30, 50, 75, 100], 'max_features': [2, 3, 4], 'max_depth': [None, 2, 3, 4, 5]}
grid_rand_for_auc = GridSearchCV(rand_for, param_grid = grid_values, scoring = 'roc_auc')
grid_rand_for_auc.fit(X_e_train, y_e_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=300,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [15, 30, 50, 75, 100], 'max_features': [2, 3, 4], 'max_depth': [None, 2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [18]:
# Best ROC score:
grid_rand_for_auc.best_params_

{'max_depth': 5, 'max_features': 2, 'n_estimators': 100}

In [19]:
# Best parameters:
grid_rand_for_auc.best_score_

0.91128738312845403

In [20]:
# Training Random Forest model with best parameters:
random_forest_e = RandomForestClassifier(max_depth = 5, max_features = 2, n_estimators = 100, random_state = 300)
random_forest_e.fit(X_e_train, y_e_train)

# Evaluating the model with test set:
y_e_pred_test = random_forest_e.predict(X_e_test)
roc_auc_e = roc_auc_score(y_e_test, y_e_pred_test)

print('Roc AUC score for test set: '+ str(round(roc_auc_e, 2)))

Roc AUC score for test set: 0.79


<p>
    Finally, we are making link prediction for the edges:
</p>

In [21]:
# Making probability prediction for missing Future Connection:
X_e_pred = df_e_pred[e_feat_columns]
y_e_pred = random_forest_e.predict_proba(X_e_pred)

# Creating a pandas Series indicating the edge's probability of happening (two employees create a connection):
prob_e_list = [prob_e[1] for prob_e in y_e_pred]
fut_conn_prob = pd.Series(data = prob_e_list, index = df_e_pred.index.values)

fut_conn_prob

(0, 9)          0.058250
(0, 19)         0.093424
(0, 20)         0.248937
(0, 35)         0.016786
(0, 38)         0.013170
(0, 42)         0.819971
(0, 43)         0.019588
(0, 50)         0.013088
(0, 53)         0.667055
(0, 54)         0.066407
(0, 62)         0.446667
(0, 63)         0.073693
(0, 69)         0.026623
(0, 72)         0.013088
(0, 83)         0.188604
(0, 90)         0.031126
(0, 91)         0.016883
(0, 95)         0.140424
(0, 100)        0.044381
(0, 106)        0.970887
(0, 108)        0.016870
(0, 109)        0.013088
(0, 110)        0.013088
(0, 118)        0.013170
(0, 122)        0.032400
(0, 131)        0.067190
(0, 133)        0.719686
(0, 140)        0.071822
(0, 149)        0.295241
(0, 151)        0.013170
                  ...   
(988, 989)      0.013088
(988, 996)      0.013088
(988, 997)      0.060013
(988, 1000)     0.013088
(988, 1002)     0.013088
(989, 994)      0.013088
(989, 996)      0.013088
(989, 1003)     0.013088
(989, 1004)     0.013088
